# get validation, n downsample indices

In [1]:
import json
import os
import gc
import numpy as np
import pandas as pd

def downsampling(positive_idx, negative_idx, ratio):
    idx = np.random.choice(negative_idx, int(ratio*len(negative_idx)))
    idx = np.concatenate((idx, positive_idx))
    return np.sort(idx).astype(int).tolist()


PATH1 = '/home/kai/data/kaggle/talkingdata/data/'
PATH0 = '/home/kai/data/kaggle/talkingdata/haoyandata1/'
PATH = '/home/kai/data/kaggle/talkingdata/downsampling/'

file = {}
train = pd.read_feather(PATH0 + 'train_cleaned.ftr')
# train = pd.read_csv(PATH1 + 'train_cleaned.csv', nrows=10000)
val_idx = (train['day'] == 9) & (train['hour'] == 13) | (train['hour'] == 17) | (train['hour'] == 21)
file['val_idx'] = val_idx[val_idx].index.values.astype(int).tolist()


# Downsampling
train = train[~val_idx]
negative = list(train[train['is_attributed']==0].index.values)
positive = list(train[train['is_attributed']==1].index.values)
ratio = len(positive)/len(negative)
idx_list = []
for i in range(5):
    idx_list.append(downsampling(positive, negative, ratio))
    print(i)
file['downsampling'] = idx_list
with open(PATH + 'idx_file.json', 'w') as f:
    json.dump(file, f)
print('done')

# split files into several by validation and down sample indices

In [54]:
import os
import gc
import json
import pandas as pd

ORIGINAL_FILE_PATH = '/home/kai/data/kaggle/talkingdata/haoyandata1/'
DOWNSAMPLING_PATH = '/home/kai/data/kaggle/talkingdata/downsampling/'
suffix = 'ftr'
with open(DOWNSAMPLING_PATH + 'idx_file.json', 'r') as f:
    data = json.load(f)
    val_idx = data['val_idx']
    downsampling_idx = data['downsampling']

print('making directory')
directory = os.path.dirname((DOWNSAMPLING_PATH + 'val/'))
if not os.path.exists(directory):
    os.makedirs(directory)

for i in range(len(downsampling_idx)):
    directory = os.path.dirname((DOWNSAMPLING_PATH + str(i) + '/'))
    if not os.path.exists(directory):
        os.makedirs(directory)
    
print('down sampling')
for file in os.listdir(ORIGINAL_FILE_PATH):
    if file.split('.')[-1] == suffix:
        data = pd.read_feather(ORIGINAL_FILE_PATH + file)
        # validation
        data.loc[val_idx].reset_index(drop=True).to_feather(DOWNSAMPLING_PATH + 'val/' + file)
        # down sampling
        for i, x in enumerate(downsampling_idx):
            data.loc[x].reset_index(drop=True).to_feather(DOWNSAMPLING_PATH + str(i) + '/' + file)
        del data
        print(file)
        gc.collect()

making directory
down sampling
train__cumcount_ip_day.ftr


# concatenate val, down sampling files into one

In [2]:
import os
import gc
import pandas as pd

PATH = '/home/kai/data/kaggle/talkingdata/downsampling/'
suffix = 'ftr'
dir_list = ['val/', '1/', '2/', '3/', '4/', '0/']
for x in dir_list:
    CURRENT_PATH = PATH + x
    file_name = PATH + x[:-1] + '.' + suffix
    df = pd.read_feather(CURRENT_PATH + 'train_cleaned.ftr')
    for y in os.listdir(CURRENT_PATH):
        splitted_y = y.split('.')
        if splitted_y[0] != 'train_cleaned' and splitted_y[1] == suffix:
            tmp = pd.read_feather(CURRENT_PATH + y)
            df[splitted_y[0].split('__')[1]] = tmp
            del tmp
    print('converting')
    df.to_feather(file_name)
    del df
    gc.collect()
    print(file_name)

converting
/home/kai/data/kaggle/talkingdata/downsampling/val.ftr
converting
/home/kai/data/kaggle/talkingdata/downsampling/1.ftr
converting
/home/kai/data/kaggle/talkingdata/downsampling/2.ftr
converting
/home/kai/data/kaggle/talkingdata/downsampling/3.ftr
converting
/home/kai/data/kaggle/talkingdata/downsampling/4.ftr
converting
/home/kai/data/kaggle/talkingdata/downsampling/0.ftr


# test concate

In [51]:
import pandas as pd
import os
import gc

PATH = '/home/kai/data/kaggle/talkingdata/haoyandata1/'
file_name = '/home/kai/data/kaggle/talkingdata/downsampling/test_cleaned.ftr'
df = pd.read_feather(PATH + 'test_cleaned.ftr')
for x in os.listdir(PATH):
    splitted = x.split('__')
    if splitted[0] == 'test_supplement':
        print(x)
        tmp = pd.read_feather(PATH + x)
        df[splitted[1].split('.')[0]] = tmp
        del tmp
        gc.collect()
print('converting')
df.to_feather(file_name)
del df
gc.collect()
print(file_name)

test_supplement__nunique_day_ip_app.ftr
test_supplement__woe_ip_channel_nextClickLeakDayFlt.ftr
test_supplement__woe_app_device_nextClickLeakDayFlt.ftr
test_supplement__variance_ip_device_hour.ftr
test_supplement__woe_ip_app.ftr
test_supplement__count_ip_app_os_channel_day.ftr
test_supplement__woe_ip_nextClickLeakDayFlt.ftr
test_supplement__com1_ip.ftr
test_supplement__woe_app_channel.ftr
test_supplement__woe_ip_app_device_os.ftr
test_supplement__nunique_day_ip_machine.ftr
test_supplement__rang_count_ip_device_os_dayhourminute.ftr
test_supplement__count_ratio_channel_app.ftr
test_supplement__woe_os_channel_nextClickLeakDayFlt.ftr
test_supplement__woe_app_device_channel.ftr
test_supplement__woe_os_nextClickLeakDayFlt.ftr
test_supplement__woe_channel.ftr
test_supplement__count_ip.ftr
test_supplement__woe_app_device.ftr
test_supplement__woe_app_os.ftr
test_supplement__woe_app_device_os.ftr
test_supplement__woe_ip_device_os.ftr
test_supplement__woe_app_device_os_channel.ftr
test_supplement

# test supplement to test

In [ ]:
import pandas as pd
t_supp = pd.read_feather('/home/kai/data/kaggle/talkingdata/downsampling/test_cleaned.ftr')


# name change: train(test_supplement)__feature_name.ftr

In [3]:
import os

PATH = '/home/kai/data/kaggle/talkingdata/haoyandata1/'

for file in os.listdir(PATH):
    tmp = file.split('.')
    if tmp[0] != 'train_cleaned' and tmp[0] != 'test_cleaned' and tmp[1] == 'ftr':
        ttmp = tmp[0].split('_')
        if ttmp[0] == 'train':
            nfile = 'train__' + '_'.join(ttmp[1:]) + '.ftr'
        else:
            nfile = '_'.join(ttmp[:2]) + '__' + '_'.join(ttmp[2:]) + '.ftr'
        os.rename(PATH + file, PATH + nfile)
        print(file, '------>', nfile)

train_count_app_device_day_hour.ftr train__count_app_device_day_hour.ftr
test_supplement_woe_app_os.ftr test_supplement__woe_app_os.ftr
test_supplement_woe_device.ftr test_supplement__woe_device.ftr
train_woe_ip_os_nextClickLeakDayFlt.ftr train__woe_ip_os_nextClickLeakDayFlt.ftr
test_supplement_woe_app_channel.ftr test_supplement__woe_app_channel.ftr
train_rang_count_ip_device_os_dayhourminute.ftr train__rang_count_ip_device_os_dayhourminute.ftr
test_supplement_woe_channel.ftr test_supplement__woe_channel.ftr
train_rang_count_ip_channel_dayhourminute.ftr train__rang_count_ip_channel_dayhourminute.ftr
train_count_ip_app_device_os_day_hour.ftr train__count_ip_app_device_os_day_hour.ftr
train_rang_count_ip_channel_dayhourminute10.ftr train__rang_count_ip_channel_dayhourminute10.ftr
train_variance_ip_device_hour.ftr train__variance_ip_device_hour.ftr
test_supplement_rang_count_ip_day.ftr test_supplement__rang_count_ip_day.ftr
test_supplement_rang_count_ip_hour.ftr test_supplement__rang_cou

# delete all test_supplement files in downsampling

In [14]:
import os

PATH = '/home/kai/data/kaggle/talkingdata/downsampling/'
dir_list = ['val/','1/','2/','3/','4/','0/']
for x in dir_list:
    for y in os.listdir(PATH + x):
        if y.split('__')[0] == 'test_supplement':
            os.remove(PATH + x + y)
            print(PATH + x + y)

/home/kai/data/kaggle/talkingdata/downsampling/val/test_supplement__nunique_day_ip_app.ftr
/home/kai/data/kaggle/talkingdata/downsampling/val/test_supplement__woe_ip_channel_nextClickLeakDayFlt.ftr
/home/kai/data/kaggle/talkingdata/downsampling/val/test_supplement__woe_app_device_nextClickLeakDayFlt.ftr
/home/kai/data/kaggle/talkingdata/downsampling/val/test_supplement__variance_ip_device_hour.ftr
/home/kai/data/kaggle/talkingdata/downsampling/val/test_supplement__woe_ip_app.ftr
/home/kai/data/kaggle/talkingdata/downsampling/val/test_supplement__count_ip_app_os_channel_day.ftr
/home/kai/data/kaggle/talkingdata/downsampling/val/test_supplement__woe_ip_nextClickLeakDayFlt.ftr
/home/kai/data/kaggle/talkingdata/downsampling/val/test_supplement__com1_ip.ftr
/home/kai/data/kaggle/talkingdata/downsampling/val/test_supplement__woe_app_channel.ftr
/home/kai/data/kaggle/talkingdata/downsampling/val/test_supplement__woe_ip_app_device_os.ftr
/home/kai/data/kaggle/talkingdata/downsampling/val/test_

/home/kai/data/kaggle/talkingdata/downsampling/1/test_supplement__nunique_machine_app.ftr
/home/kai/data/kaggle/talkingdata/downsampling/1/test_supplement__woe_ip_app_os.ftr
/home/kai/data/kaggle/talkingdata/downsampling/1/test_supplement__nunique_day_ip_device.ftr
/home/kai/data/kaggle/talkingdata/downsampling/1/test_supplement__woe_ip_device_os_channel.ftr
/home/kai/data/kaggle/talkingdata/downsampling/1/test_supplement__woe_os.ftr
/home/kai/data/kaggle/talkingdata/downsampling/1/test_supplement__count_ip_device_os_day_hourminute10.ftr
/home/kai/data/kaggle/talkingdata/downsampling/1/test_supplement__count_ip_app_os_channel.ftr
/home/kai/data/kaggle/talkingdata/downsampling/1/test_supplement__count_ratio_ip_channel.ftr
/home/kai/data/kaggle/talkingdata/downsampling/1/test_supplement__woe_ip_device.ftr
/home/kai/data/kaggle/talkingdata/downsampling/1/test_supplement__woe_ip_app_device_channel.ftr
/home/kai/data/kaggle/talkingdata/downsampling/2/test_supplement__nunique_day_ip_app.ftr
